# Imports

In [3]:
# General
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Hyperparameter tuning
import optuna

# Models
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC


# Data processing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Metrics
from sklearn.metrics import roc_auc_score

# Data preparation

In [4]:
# Data input
data = pd.read_csv('../data/train.csv')
data_to_predict = pd.read_csv('../data/test.csv')
data_ccrisk = pd.read_csv('../data/credit_risk_dataset.csv')

X_old = data.drop(['loan_status'], axis=1)
y_old = data['loan_status']

# Merge the dataframes
data_no_id = data.drop(['id'], axis=1)
merged_data = pd.concat([data_no_id, data_ccrisk], ignore_index=True)

X = merged_data.drop(['loan_status'], axis=1)
y = merged_data['loan_status']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, shuffle=True, test_size=0.2, random_state=2024)

In [5]:
# Data processing

categorical_columns = X.select_dtypes(include=['object']).columns

categorical_ordinal = ['loan_grade']
categorical_onehot = categorical_columns.drop(categorical_ordinal)

numerical_columns = X.select_dtypes(include=['int64', 'float64']).columns

# Preprocessor
preprocessor = ColumnTransformer(
	transformers=[
		('ordinal', OrdinalEncoder(), categorical_ordinal),
		('onehot', OneHotEncoder(), categorical_onehot),
		('scaler', StandardScaler(), numerical_columns)
	])

preprocessor.fit(X_train)

X_train_prep = preprocessor.transform(X_train)
X_test_prep = preprocessor.transform(X_test)

# Models

## XGBoost

In [6]:
def xgb_objective(trial):
    # Hyperparameter search space
    param = {
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 0.5),
        'lambda': trial.suggest_float('lambda', 1e-8, 10.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 10.0, log=True),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 3.0),
        'eval_metric': 'auc',  # AUC is the metric
        'objective': 'binary:logistic'  # Use binary logistic, probability outputs
    }

    # Initialize the model
    model = xgb.XGBClassifier(**param)

    # Fit the model
    model.fit(X_train_prep, y_train, 
              eval_set=[(X_test_prep, y_test)],  
              verbose=False)

    # Predict proba for validation set
    y_pred_prob = model.predict_proba(X_test_prep)[:, 1]

    auc = roc_auc_score(y_test, y_pred_prob)
    
    return auc

# Study object to run the optimization. I want to maximize AUC
xgb_study = optuna.create_study(direction='maximize')
xgb_study.optimize(xgb_objective, n_trials=50)


print(f"Best trial: {xgb_study.best_trial.params}")

[I 2024-10-09 22:05:58,334] A new study created in memory with name: no-name-44073533-931b-4bf9-a0e4-6b809fdcc510
[I 2024-10-09 22:06:01,537] Trial 0 finished with value: 0.9525810347956751 and parameters: {'max_depth': 8, 'learning_rate': 0.23759751295096465, 'n_estimators': 104, 'subsample': 0.6861085235631791, 'colsample_bytree': 0.6075327980311174, 'gamma': 0.12451478473153665, 'lambda': 1.5462291489103863e-06, 'alpha': 0.0014761980421400247, 'scale_pos_weight': 2.9166529779731913}. Best is trial 0 with value: 0.9525810347956751.
[I 2024-10-09 22:06:12,725] Trial 1 finished with value: 0.9548030154749317 and parameters: {'max_depth': 10, 'learning_rate': 0.10771767717037606, 'n_estimators': 374, 'subsample': 0.7953274560696989, 'colsample_bytree': 0.8337629717071376, 'gamma': 0.2995102374207908, 'lambda': 1.2907977806555766, 'alpha': 1.3013028313152889e-06, 'scale_pos_weight': 2.323853613045628}. Best is trial 1 with value: 0.9548030154749317.
[I 2024-10-09 22:06:20,947] Trial 2 fi

Best trial: {'max_depth': 9, 'learning_rate': 0.04443861243064845, 'n_estimators': 295, 'subsample': 0.9129336231196766, 'colsample_bytree': 0.7817900881520917, 'gamma': 0.20733828465084803, 'lambda': 0.0007763279982043077, 'alpha': 4.17984861400515, 'scale_pos_weight': 2.6715664022423775}


In [7]:
xgb_best_params = xgb_study.best_trial.params
xgb_best_score = xgb_study.best_trial.value
xgb_best_score

0.959733166826617

## CatBoostClassifier

In [8]:
from catboost import CatBoostClassifier

In [9]:
def cat_objective(trial):
    # Define the hyperparameter search space
    param = {
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'iterations': trial.suggest_int('iterations', 200, 1000),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True),
        'border_count': trial.suggest_int('border_count', 32, 255),  # CatBoost feature binning
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'random_strength': trial.suggest_float('random_strength', 0.0, 1.0),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 3.0),
        'eval_metric': 'AUC',  # Using AUC as the evaluation metric
        'logging_level': 'Silent',  # Disable CatBoost output
        'task_type': 'CPU',  # Use CPU (set 'GPU' if you have a GPU available)
        'use_best_model': True
    }

    # Initialize the CatBoost model with the trial's parameters
    model = CatBoostClassifier(**param)

    # Fit the model
    model.fit(X_train_prep, y_train,
              eval_set=[(X_test_prep, y_test)],
              early_stopping_rounds=20,  # Early stopping to prevent overfitting
              verbose=False)

    # Get predictions and calculate AUC score
    y_pred_prob = model.predict_proba(X_test_prep)[:, 1]  # Probabilities for class 1
    auc = roc_auc_score(y_test, y_pred_prob)

    return auc

# Create a study object and start the optimization process
cat_study = optuna.create_study(direction='maximize')  # We want to maximize AUC
cat_study.optimize(cat_objective, n_trials=50)

# Output the best trial
print(f"Best trial: {cat_study.best_trial.params}")

[I 2024-10-09 22:11:51,100] A new study created in memory with name: no-name-e72c9759-8c17-475d-a66c-405ea194cf53
[I 2024-10-09 22:11:58,468] Trial 0 finished with value: 0.9535014045536119 and parameters: {'depth': 7, 'learning_rate': 0.14900669887451898, 'iterations': 972, 'l2_leaf_reg': 2.832927821560049e-06, 'border_count': 164, 'bagging_temperature': 0.5177074086756733, 'random_strength': 0.3755586295324942, 'scale_pos_weight': 1.0033482236176379}. Best is trial 0 with value: 0.9535014045536119.
[I 2024-10-09 22:12:01,653] Trial 1 finished with value: 0.9435656688784282 and parameters: {'depth': 9, 'learning_rate': 0.1442107216774723, 'iterations': 403, 'l2_leaf_reg': 1.9701502555594304e-05, 'border_count': 91, 'bagging_temperature': 0.14870006576346828, 'random_strength': 0.12286492442004282, 'scale_pos_weight': 2.431117089849309}. Best is trial 0 with value: 0.9535014045536119.
[I 2024-10-09 22:12:11,152] Trial 2 finished with value: 0.9434425794273231 and parameters: {'depth': 

Best trial: {'depth': 6, 'learning_rate': 0.1377846993488999, 'iterations': 335, 'l2_leaf_reg': 1.9947064092180422, 'border_count': 223, 'bagging_temperature': 0.43873437022890877, 'random_strength': 0.3977615491895288, 'scale_pos_weight': 2.4634007576197803}


In [10]:
cat_best_params = cat_study.best_trial.params
cat_best_score = cat_study.best_trial.value
cat_best_score

0.9595616312456452

In [12]:
# Objective function for LightGBM
def lgb_objective(trial):
    param = {
        'objective': 'binary',
        'metric': 'auc',
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'min_child_weight': trial.suggest_float('min_child_weight', 1e-3, 10.0),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
    }

    lgb_model = lgb.LGBMClassifier(**param)
    
    # Train the model
    lgb_model.fit(X_train_prep, y_train,
                  eval_set=[(X_test_prep, y_test)],
                  eval_metric='auc'
                  )

    # Predict and evaluate AUC
    y_pred_prob = lgb_model.predict_proba(X_test_prep)[:, 1]
    auc = roc_auc_score(y_test, y_pred_prob)

    return auc

# Optimize the objective function
lgb_study = optuna.create_study(direction='maximize')
lgb_study.optimize(lgb_objective, n_trials=50)

# Best parameters and AUC
print("Best LGBM Params: ", lgb_study.best_trial.params)
print("Best AUC for LGBM: ", lgb_study.best_value)

[I 2024-10-09 22:47:59,952] A new study created in memory with name: no-name-e4df7c3a-673f-47c0-a4ee-3f61b705cfdd


[LightGBM] [Info] Number of positive: 12366, number of negative: 60614
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004010 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 954
[LightGBM] [Info] Number of data points in the train set: 72980, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.169444 -> initscore=-1.589575
[LightGBM] [Info] Start training from score -1.589575
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

[I 2024-10-09 22:48:06,809] Trial 0 finished with value: 0.9605070990867199 and parameters: {'learning_rate': 0.20286923205339508, 'num_leaves': 268, 'max_depth': 3, 'min_child_samples': 64, 'min_child_weight': 5.427852638749153, 'subsample': 0.7080406522259487, 'colsample_bytree': 0.6149863635429692, 'n_estimators': 882}. Best is trial 0 with value: 0.9605070990867199.


[LightGBM] [Info] Number of positive: 12366, number of negative: 60614
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009931 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 954
[LightGBM] [Info] Number of data points in the train set: 72980, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.169444 -> initscore=-1.589575
[LightGBM] [Info] Start training from score -1.589575
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


[I 2024-10-09 22:48:10,991] Trial 1 finished with value: 0.9548021724695881 and parameters: {'learning_rate': 0.2920295183776929, 'num_leaves': 56, 'max_depth': 15, 'min_child_samples': 30, 'min_child_weight': 1.7023084792952516, 'subsample': 0.9259109299444939, 'colsample_bytree': 0.5835249499904873, 'n_estimators': 406}. Best is trial 0 with value: 0.9605070990867199.


...

[LightGBM] [Info] Number of positive: 12366, number of negative: 60614
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011911 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 954
[LightGBM] [Info] Number of data points in the train set: 72980, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.169444 -> initscore=-1.589575
[LightGBM] [Info] Start training from score -1.589575
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

[I 2024-10-09 22:52:31,914] Trial 48 finished with value: 0.9624465235825518 and parameters: {'learning_rate': 0.05625929357298218, 'num_leaves': 287, 'max_depth': 8, 'min_child_samples': 36, 'min_child_weight': 6.1290178895068035, 'subsample': 0.8069805580833423, 'colsample_bytree': 0.6288177374771741, 'n_estimators': 706}. Best is trial 46 with value: 0.9627544019391427.


[LightGBM] [Info] Number of positive: 12366, number of negative: 60614
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008586 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 954
[LightGBM] [Info] Number of data points in the train set: 72980, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.169444 -> initscore=-1.589575
[LightGBM] [Info] Start training from score -1.589575
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

[I 2024-10-09 22:52:41,071] Trial 49 finished with value: 0.9612736790597132 and parameters: {'learning_rate': 0.018850003105030495, 'num_leaves': 243, 'max_depth': 9, 'min_child_samples': 80, 'min_child_weight': 7.2272787228664805, 'subsample': 0.8714441381979217, 'colsample_bytree': 0.6501289447353497, 'n_estimators': 648}. Best is trial 46 with value: 0.9627544019391427.


Best LGBM Params:  {'learning_rate': 0.056732075773989935, 'num_leaves': 200, 'max_depth': 8, 'min_child_samples': 81, 'min_child_weight': 4.238151660690304, 'subsample': 0.8031135623721948, 'colsample_bytree': 0.6213074649038759, 'n_estimators': 530}
Best AUC for LGBM:  0.9627544019391427


In [13]:
lgb_best_params = lgb_study.best_trial.params
lgb_best_score = lgb_study.best_value
lgb_best_score

0.9627544019391427

In [14]:
from sklearn.impute import SimpleImputer
from sklearn.neural_network import MLPClassifier

# Impute missing values
imputer = SimpleImputer(strategy='mean')  # or use 'median' depending on your data
X_train_prep = imputer.fit_transform(X_train_prep)
X_test_prep = imputer.transform(X_test_prep)

# Add MLPClassifier as one of the base models
mlp = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=500)

# Update estimators list for stacking
estimators = [
    ('catboost', CatBoostClassifier(**cat_study.best_trial.params)),
    ('lightgbm', lgb.LGBMClassifier(**lgb_study.best_trial.params)),
    ('xgb', xgb.XGBClassifier(**xgb_study.best_trial.params)),
    ('mlp', mlp)  # Add the simple neural network
]

# Define the stacking model
stacking_model = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression()  # Or RandomForest if you prefer
)

# Fit the stacking model
stacking_model.fit(X_train_prep, y_train)

# Evaluate AUC on the test set
y_pred_prob = stacking_model.predict_proba(X_test_prep)[:, 1]
stacking_auc = roc_auc_score(y_test, y_pred_prob)

print(f"Stacking AUC with MLP: {stacking_auc}")


0:	learn: 0.5751075	total: 24.2ms	remaining: 8.08s
1:	learn: 0.4950226	total: 50.8ms	remaining: 8.45s
2:	learn: 0.4384848	total: 79.4ms	remaining: 8.79s
3:	learn: 0.4027386	total: 103ms	remaining: 8.49s
4:	learn: 0.3746160	total: 130ms	remaining: 8.59s
5:	learn: 0.3566968	total: 154ms	remaining: 8.42s
6:	learn: 0.3438022	total: 179ms	remaining: 8.41s
7:	learn: 0.3328342	total: 204ms	remaining: 8.34s
8:	learn: 0.3228014	total: 234ms	remaining: 8.46s
9:	learn: 0.3145417	total: 264ms	remaining: 8.59s
10:	learn: 0.3067987	total: 294ms	remaining: 8.64s
11:	learn: 0.3031934	total: 317ms	remaining: 8.54s
12:	learn: 0.2992174	total: 347ms	remaining: 8.6s
13:	learn: 0.2941149	total: 376ms	remaining: 8.62s
14:	learn: 0.2914530	total: 401ms	remaining: 8.55s
15:	learn: 0.2886530	total: 429ms	remaining: 8.55s
16:	learn: 0.2866313	total: 461ms	remaining: 8.62s
17:	learn: 0.2847274	total: 490ms	remaining: 8.63s
18:	learn: 0.2831081	total: 516ms	remaining: 8.57s
19:	learn: 0.2821727	total: 544ms	remai

In [15]:
# Fit on the full training data
X_prep = preprocessor.transform(X)
X_prep = imputer.transform(X_prep)

# Fit the stacking model on the full data
stacking_model.fit(X_prep, y)

0:	learn: 0.5754079	total: 30.6ms	remaining: 10.2s
1:	learn: 0.4919787	total: 57.3ms	remaining: 9.54s
2:	learn: 0.4377538	total: 92.4ms	remaining: 10.2s
3:	learn: 0.3999983	total: 120ms	remaining: 9.97s
4:	learn: 0.3755795	total: 153ms	remaining: 10.1s
5:	learn: 0.3563123	total: 189ms	remaining: 10.3s
6:	learn: 0.3413232	total: 222ms	remaining: 10.4s
7:	learn: 0.3284950	total: 268ms	remaining: 11s
8:	learn: 0.3193504	total: 329ms	remaining: 11.9s
9:	learn: 0.3126755	total: 371ms	remaining: 12s
10:	learn: 0.3077524	total: 414ms	remaining: 12.2s
11:	learn: 0.3031319	total: 449ms	remaining: 12.1s
12:	learn: 0.2989435	total: 485ms	remaining: 12s
13:	learn: 0.2953847	total: 531ms	remaining: 12.2s
14:	learn: 0.2931042	total: 578ms	remaining: 12.3s
15:	learn: 0.2905887	total: 618ms	remaining: 12.3s
16:	learn: 0.2880323	total: 650ms	remaining: 12.2s
17:	learn: 0.2842813	total: 691ms	remaining: 12.2s
18:	learn: 0.2827286	total: 736ms	remaining: 12.2s
19:	learn: 0.2813449	total: 778ms	remaining:

StackingClassifier(estimators=[('catboost',
                                <catboost.core.CatBoostClassifier object at 0x0000021C23786420>),
                               ('lightgbm',
                                LGBMClassifier(colsample_bytree=0.6213074649038759,
                                               learning_rate=0.056732075773989935,
                                               max_depth=8,
                                               min_child_samples=81,
                                               min_child_weight=4.238151660690304,
                                               n_estimators=530, num_leaves=200,
                                               subsample=0.8031135623721948)),
                               ('xgb',
                                XGBClassi...
                                              learning_rate=0.04443861243064845,
                                              max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None, max_depth=9,
                                              max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=295, n_jobs=None, ...)),
                               ('mlp',
                                MLPClassifier(hidden_layer_sizes=(64, 32),
                                              max_iter=500))],
                   final_estimator=LogisticRegression())

In [ ]:
estimators_RF = [
    ('catboost', CatBoostClassifier(**cat_study.best_trial.params)),
    ('lightgbm', lgb.LGBMClassifier(**lgb_study.best_trial.params)),
    ('xgb', xgb.XGBClassifier(**xgb_study.best_trial.params)),
    ('mlp', mlp)  # Add the simple neural network
]

# Define the stacking model
stacking_model_RF = StackingClassifier(
    estimators=estimators,
    final_estimator=RandomForestClassifier()
)
# Fit the stacking model
stacking_model.fit(X_train_prep, y_train)

# Evaluate AUC on the test set
y_pred_prob_rf = stacking_model.predict_proba(X_test_prep)[:, 1]
stacking_auc_rf = roc_auc_score(y_test, y_pred_prob_rf)

print(f"Stacking AUC with MLP and RF: {stacking_auc_rf}")

0:	learn: 0.5751075	total: 23.5ms	remaining: 7.84s
1:	learn: 0.4950226	total: 46.9ms	remaining: 7.81s
2:	learn: 0.4384848	total: 69.3ms	remaining: 7.67s
3:	learn: 0.4027386	total: 97ms	remaining: 8.03s
4:	learn: 0.3746160	total: 119ms	remaining: 7.87s
5:	learn: 0.3566968	total: 143ms	remaining: 7.86s
6:	learn: 0.3438022	total: 166ms	remaining: 7.78s
7:	learn: 0.3328342	total: 195ms	remaining: 7.97s
8:	learn: 0.3228014	total: 217ms	remaining: 7.87s
9:	learn: 0.3145417	total: 244ms	remaining: 7.94s
10:	learn: 0.3067987	total: 280ms	remaining: 8.24s
11:	learn: 0.3031934	total: 314ms	remaining: 8.45s
12:	learn: 0.2992174	total: 342ms	remaining: 8.46s
13:	learn: 0.2941149	total: 372ms	remaining: 8.54s
14:	learn: 0.2914530	total: 397ms	remaining: 8.47s
15:	learn: 0.2886530	total: 423ms	remaining: 8.44s
16:	learn: 0.2866313	total: 447ms	remaining: 8.36s
17:	learn: 0.2847274	total: 472ms	remaining: 8.3s
18:	learn: 0.2831081	total: 508ms	remaining: 8.45s
19:	learn: 0.2821727	total: 532ms	remain

In [16]:
# Submission data
data_to_predict_prep = preprocessor.transform(data_to_predict)
data_to_predict_prep = imputer.transform(data_to_predict_prep)

# Predict probabilities for the submission data
y_pred_prob = stacking_model.predict_proba(data_to_predict_prep)[:, 1]

# Create a submission dataframe
submission = pd.DataFrame({
	'id': data_to_predict['id'],
	'loan_status': y_pred_prob
})
submission

,id,loan_status
0,58645,0.973510
1,58646,0.029088
2,58647,0.691275
3,58648,0.021320
4,58649,0.030864
...,...,...
39093,97738,0.043283
39094,97739,0.020252
39095,97740,0.021276
39096,97741,0.173647


In [17]:
submission.to_csv('../submissions/Stack_XGB_LGB_CatB_MLP_LogReg.csv', index=False)

In [18]:
import joblib

# Save the model
joblib.dump(stacking_model, '../models/Stack_XGB_LGB_CatB_MLP_LogReg.pkl')

['../models/Stack_XGB_LGB_CatB_MLP_LogReg.pkl']